# Image Reduction for Imaging Project
This notebook will describe how I processed the images for the data we took on 2021-10-05. I first loaded all of the images into a subdirectory called `images`. Eli was kind enough to give me the files that he created containing all of the image lists. They are listed below.

In [1]:
ls *.txt

dark_list.txt      flat_r_list.txt    object_b_list.txt  object_v_list.txt
flat_b_list.txt    flat_v_list.txt    object_r_list.txt  zero_list.txt


In order to process the data we will

1. Create a master bias frame called `Zero.fits`
2. Create a master dark current frame called `Dark.fits`
3. Create master efficiency map frames. We need one frame for each filter. They will be called `Flat_B.fits`, `Flat_V.fits`, and `Flat_R.fits`.
4. Run `ccdproc` to correct the object images. We have to do this separately for images taken with the *B*, *V*, and *R* filters.

I will also have **DS9** open on my desktop to examine the images after each step to make sure the process worked correctly. I find this easier than displaying the image using `imshow()`.

I start by importing the needed modules

In [2]:
import ccdred_sb as cc

## Create master bias frame
We will use `cc.zerocombine()`. Lets use `help()` to understand its arguments and how it works.

In [3]:
help(cc.zerocombine)

Help on function zerocombine in module ccdred_sb:

zerocombine(imdir='images/', input_list='zero_list.txt', output='Zero')
    Like IRAF's `zerocombine`. It currently combines the frames by
    averaging all of the bias frames listed in the `input_list`. It writes
    out a FITS file with the name given by the `output` parameter. Prints the
    `mean` and `sigma` (standard deviation) of the output file's pixel values.
    
    Parameters
    ----------
    imdir : str
        Path to image directory (must include trailing '/')
    input_list : str
        Name of file containing list of filenames
    output : str
        Name for the output FITS file.
    
    Returns
    -------
    fout_name : str
        Name with `.fits` extension of created file



We won't have to change any of the default arguments since all of our images are in the `images` directory and the list of bias frames is called `zero_list.txt`.

In [4]:
cc.zerocombine()

Reading 20211005_043.fits
Reading 20211005_044.fits
Reading 20211005_045.fits
Reading 20211005_046.fits
Reading 20211005_047.fits
Reading 20211005_048.fits
Reading 20211005_049.fits
Reading 20211005_050.fits
Reading 20211005_051.fits
Reading 20211005_042.fits
zerocombine output file name: Zero.fits
    mean pixel value = 924.462668
    standard deviation = 3.877027


'Zero.fits'

I examined the image using **DS9** and it looks like what I expected and the signal level is about 942. This is also what I expected. 

## Create master dark current image
Again I use `help(cc.darkcombine)` to make sure I have the `cc.darkcombine` function's parameters are set correctly.

In [5]:
help(cc.darkcombine)

Help on function darkcombine in module ccdred_sb:

darkcombine(imdir='images/', input_list='dark_list.txt', output='Dark')
    Like IRAF's `darkcombine`. Creates dark current images and then it
    currently combines the frames by averaging all of the dark current
    frames listed in the `input_list`. It writes out a FITS file with the name
    given by the `output` parameter. 
    
    Parameters
    ----------
    imdir : str
        Path to image directory (must include trailing '/')
    input_list : str
        Name of file containing list of filenames
    output : str
        Name for the output FITS file.
    
    Returns
    -------
    fout_name : str
        Name with `.fits` extension of created file



Again, I don't need to change any of the functions parameters.

In [6]:
cc.darkcombine()

Reading 20211005_052.fits
Reading 20211005_053.fits
Reading 20211005_054.fits
Reading 20211005_055.fits
Reading 20211005_056.fits
the exposure time is 300
the exposure time is 300
the exposure time is 300
the exposure time is 300
the exposure time is 300
dark_data type <class 'numpy.ndarray'>
darkcombine output file name: Dark.fits
    mean pixel value = 3.112430
    standard deviation = 1.046245


'Dark.fits'

The dark frame looks like what I expect and the pixel values have a mean value of 3.1 pixels. We can get the mean dark current in electrons/sec by multiplying this number by the CCD gain. This information isn't written in the header under the `GAIN` keyword, but we know the number from the manufacturer is 2.2 electrons/ADU so the dark current is about 7 electrons. This is a reasonable number for this CCD cooled to -24C. 

## Create master efficiency maps
Efficiency maps are sometimes simply called flat field images. We need to create one for each filter. Let's use `help()` to determine how we need to reset parameters for the `cc.flatcombine()` function.

In [7]:
help(cc.flatcombine)

Help on function flatcombine in module ccdred_sb:

flatcombine(imdir='images/', input_list='flat_list.txt', output='Flat', zero_sub=True, zero_file='Zero.fits', dark_correction=False, dark_file='Dark.fits')
    Like IRAF's `flatcombine`. 
    
    It currently works by: 
    
    1. Subtract `zero_file` from each file in `input_list` if `zero_sub = True`
    2. Compute median pixel value for each file in `input_list`
    3. Create the master efficiency map file with name specified by `output`
    
    Parameters
    ----------
    imdir : str
        Path to image directory (must include trailing '/')
    input_list : str
        Name of file containing list of filenames
    output : str
        Name for the output FITS file.
    zero_sub : boolean
        Flag for bias subtraction
    zero_file : str
        Name of bias file for bias subraction        
    dark_correction : boolean
        Flag for dark subtraction
    dark_file : str
        Name of dark file for dark subraction    

Ther are two parameters that we must reset from their default values. We need to set the `input_list` to the list for each of the three filters, and we have to set `dark_correction` to `True` since the data from this CCD requires a dark current correction.

In [8]:
cc.flatcombine(input_list='flat_b_list.txt', dark_correction=True)

Reading 20211005_001.fits
Reading 20211005_002.fits
Reading 20211005_003.fits
Reading 20211005_004.fits
Reading 20211005_005.fits
Dark current correction with Dark.fits: 
Flat image meadian = 26822.412000
Dark current correction with Dark.fits: 
Flat image meadian = 25337.479500
Dark current correction with Dark.fits: 
Flat image meadian = 20605.644000
Dark current correction with Dark.fits: 
Flat image meadian = 22690.981667
Dark current correction with Dark.fits: 
Flat image meadian = 22871.187000
flatcombine output file name: Flat_B.fits


'Flat_B.fits'

The file looks like what I expect an efficiency map should. It's average pixel value is about 1.0 which it should be for an efficiency map.

In [9]:
cc.flatcombine(input_list='flat_v_list.txt', dark_correction=True)

Reading 20211005_006.fits
Reading 20211005_007.fits
Reading 20211005_008.fits
Reading 20211005_009.fits
Reading 20211005_010.fits
Dark current correction with Dark.fits: 
Flat image meadian = 26287.080000
Dark current correction with Dark.fits: 
Flat image meadian = 20695.264000
Dark current correction with Dark.fits: 
Flat image meadian = 20081.717333
Dark current correction with Dark.fits: 
Flat image meadian = 17208.009333
Dark current correction with Dark.fits: 
Flat image meadian = 16890.645333
flatcombine output file name: Flat_V.fits


'Flat_V.fits'

In [10]:
cc.flatcombine(input_list='flat_r_list.txt', dark_correction=True)

Reading 20211005_011.fits
Reading 20211005_012.fits
Reading 20211005_013.fits
Reading 20211005_014.fits
Reading 20211005_015.fits
Dark current correction with Dark.fits: 
Flat image meadian = 11968.940000
Dark current correction with Dark.fits: 
Flat image meadian = 14579.096000
Dark current correction with Dark.fits: 
Flat image meadian = 20582.400000
Dark current correction with Dark.fits: 
Flat image meadian = 17489.700000
Dark current correction with Dark.fits: 
Flat image meadian = 22080.425000
flatcombine output file name: Flat_R.fits


'Flat_R.fits'

By examining with **DS9** it seems that both of these images are correct as well.

# Proccess the object images
Again I start by getting help for the function.

In [11]:
help(cc.ccdproc)

Help on function ccdproc in module ccdred_sb:

ccdproc(imdir='images/', input_list='object_list.txt', output_list=None, zero_file='Zero.fits', flat_file='Flat.fits', dark_correction=False, dark_file='Dark.fits')
    Like IRAF's `ccdproc`. 
    
    It currently works by: 
    
    1. Subtract `zero_file` from each file in the `input_list` 
    2. Divide by `flat_file` from each file in the `input_list`
    3. Write processed FITS files with name specified by `output_list`
    
    Parameters
    ----------
    input_list : str
        Name of file containing list of input file names
    output_list : str
        Name of file containing output file names. If `output_list = None` the
        filenames are the same as those in `input_list` with `_p` appended.
    zero_file : str
        Name of bias correction file
    flat_file : str
        Name of flat correction file       
    dark_correction : boolean
        applies dark correction if True
    dark_file : str
        dark current i

For this function we need to change three input parameters:
1. We need to change the `input_list` filename. We have three files for the three filters.
2. We need to change `flat_file` to use the flat file for each of the three different filters.
3. We need to change `dark_correction=True` to correct the images for dark current.

In [12]:
cc.ccdproc(input_list='object_b_list.txt', flat_file='Flat_B.fits', dark_correction=True)

Running ccdproc
Reading 20211005_062.fits
Reading 20211005_063.fits
Reading 20211005_064.fits
Reading 20211005_065.fits
Reading 20211005_066.fits
Reading 20211005_067.fits
Reading 20211005_068.fits
Reading 20211005_069.fits
Reading 20211005_070.fits
Reading 20211005_071.fits
Using dark current image: Dark.fits
Using zero image: Zero.fits
Using flat image: Flat_B.fits
Using dark current image: Dark.fits
Using zero image: Zero.fits
Using flat image: Flat_B.fits
Using dark current image: Dark.fits
Using zero image: Zero.fits
Using flat image: Flat_B.fits
Using dark current image: Dark.fits
Using zero image: Zero.fits
Using flat image: Flat_B.fits
Using dark current image: Dark.fits
Using zero image: Zero.fits
Using flat image: Flat_B.fits
Using dark current image: Dark.fits
Using zero image: Zero.fits
Using flat image: Flat_B.fits
Using dark current image: Dark.fits
Using zero image: Zero.fits
Using flat image: Flat_B.fits
Using dark current image: Dark.fits
Using zero image: Zero.fits
Us

['20211005_062_p.fits',
 '20211005_063_p.fits',
 '20211005_064_p.fits',
 '20211005_065_p.fits',
 '20211005_066_p.fits',
 '20211005_067_p.fits',
 '20211005_068_p.fits',
 '20211005_069_p.fits',
 '20211005_070_p.fits',
 '20211005_071_p.fits']

In [13]:
cc.ccdproc(input_list='object_v_list.txt', flat_file='Flat_V.fits', dark_correction=True)

Running ccdproc
Reading 20211005_037.fits
Reading 20211005_038.fits
Reading 20211005_039.fits
Reading 20211005_040.fits
Reading 20211005_041.fits
Using dark current image: Dark.fits
Using zero image: Zero.fits
Using flat image: Flat_V.fits
Using dark current image: Dark.fits
Using zero image: Zero.fits
Using flat image: Flat_V.fits
Using dark current image: Dark.fits
Using zero image: Zero.fits
Using flat image: Flat_V.fits
Using dark current image: Dark.fits
Using zero image: Zero.fits
Using flat image: Flat_V.fits
Using dark current image: Dark.fits
Using zero image: Zero.fits
Using flat image: Flat_V.fits


['20211005_037_p.fits',
 '20211005_038_p.fits',
 '20211005_039_p.fits',
 '20211005_040_p.fits',
 '20211005_041_p.fits']

In [14]:
cc.ccdproc(input_list='object_r_list.txt', flat_file='Flat_R.fits', dark_correction=True)

Running ccdproc
Reading 20211005_057.fits
Reading 20211005_058.fits
Reading 20211005_059.fits
Reading 20211005_060.fits
Reading 20211005_061.fits
Using dark current image: Dark.fits
Using zero image: Zero.fits
Using flat image: Flat_R.fits
Using dark current image: Dark.fits
Using zero image: Zero.fits
Using flat image: Flat_R.fits
Using dark current image: Dark.fits
Using zero image: Zero.fits
Using flat image: Flat_R.fits
Using dark current image: Dark.fits
Using zero image: Zero.fits
Using flat image: Flat_R.fits
Using dark current image: Dark.fits
Using zero image: Zero.fits
Using flat image: Flat_R.fits


['20211005_057_p.fits',
 '20211005_058_p.fits',
 '20211005_059_p.fits',
 '20211005_060_p.fits',
 '20211005_061_p.fits']

## Summary
The final step is to examine the images to make sure they look reasonable. The next step is to see if there is any misalignments between the images and if there are align the images so we can sum them to get the best signal-to-noise ratio in the final image. Once we have the aligned and combined images we can use **DS9** to create color images of the objects.